<center>
    <p style="text-align:center">
        <img alt="phoenix logo" src="https://raw.githubusercontent.com/Arize-ai/phoenix-assets/9e6101d95936f4bd4d390efc9ce646dc6937fb2d/images/socal/github-large-banner-phoenix.jpg" width="1000"/>
        <br>
        <br>
        <a href="https://docs.arize.com/phoenix/">Docs</a>
        |
        <a href="https://github.com/Arize-ai/phoenix">GitHub</a>
        |
        <a href="https://join.slack.com/t/arize-ai/shared_invite/zt-1px8dcmlf-fmThhDFD_V_48oU7ALan4Q">Community</a>
    </p>
</center>
<h1 align="center">Tracing CrewAI with Arize Phoenix - Parallelization Workflow</h1>

In [11]:
%pip install -q arize-phoenix opentelemetry-sdk opentelemetry-exporter-otlp crewai crewai_tools openinference-instrumentation-crewai

Note: you may need to restart the kernel to use updated packages.


# Set up Keys and Dependencies

Note: For this colab you'll need:

*   OpenAI API key (https://openai.com/)
*   Serper API key (https://serper.dev/)
*   Phoenix API key (https://app.phoenix.arize.com/)

In [12]:
import getpass
import os

# Prompt the user for their API keys if they haven't been set
openai_key = os.getenv("OPENAI_API_KEY", "OPENAI_API_KEY")
serper_key = os.getenv("SERPER_API_KEY", "SERPER_API_KEY")

if openai_key == "OPENAI_API_KEY":
    openai_key = getpass.getpass("Please enter your OPENAI_API_KEY: ")

if serper_key == "SERPER_API_KEY":
    serper_key = getpass.getpass("Please enter your SERPER_API_KEY: ")

# Set the environment variables with the provided keys
os.environ["OPENAI_API_KEY"] = openai_key
os.environ["SERPER_API_KEY"] = serper_key

if "PHOENIX_API_KEY" not in os.environ:
    os.environ["PHOENIX_API_KEY"] = getpass.getpass("Enter your Phoenix API key: ")

os.environ["PHOENIX_CLIENT_HEADERS"] = f"api_key={os.environ['PHOENIX_API_KEY']}"
os.environ["PHOENIX_COLLECTOR_ENDPOINT"] = "https://app.phoenix.arize.com/"

## Configure Tracing

In [13]:
from phoenix.otel import register

tracer_provider = register(
    project_name="crewai-agents", endpoint="https://app.phoenix.arize.com/v1/traces"
)

Overriding of current TracerProvider is not allowed


🔭 OpenTelemetry Tracing Details 🔭
|  Phoenix Project: crewai-agents
|  Span Processor: SimpleSpanProcessor
|  Collector Endpoint: https://app.phoenix.arize.com/v1/traces
|  Transport: HTTP + protobuf
|  Transport Headers: {'api_key': '****', 'authorization': '****'}
|  
|  Using a default SpanProcessor. `add_span_processor` will overwrite this default.
|  
|  ⚠️ WARNING: It is strongly advised to use a BatchSpanProcessor in production environments.
|  
|  `register` has set this TracerProvider as the global OpenTelemetry default.
|  To disable this behavior, call `register` with `set_global_tracer_provider=False`.



# Instrument CrewAI

In [14]:
from openinference.instrumentation.crewai import CrewAIInstrumentor

CrewAIInstrumentor().instrument(skip_dep_check=True, tracer_provider=tracer_provider)

Attempting to instrument while already instrumented


## Define your Agents

In [15]:
from crewai import Agent, Crew, Task
from crewai.process import Process

researcher_1 = Agent(
    role="LLM Researcher A",
    goal="Research trend #1 in AI and summarize it clearly.",
    backstory="Specializes in model safety and governance.",
    verbose=True,
)

researcher_2 = Agent(
    role="LLM Researcher B",
    goal="Research trend #2 in AI and summarize it clearly.",
    backstory="Expert in multimodal and frontier models.",
    verbose=True,
)

researcher_3 = Agent(
    role="LLM Researcher C",
    goal="Research trend #3 in AI and summarize it clearly.",
    backstory="Focused on AI policy and alignment.",
    verbose=True,
)

aggregator = Agent(
    role="Aggregator",
    goal="Combine and synthesize all research into a single summary report.",
    backstory="Information architect skilled at summarizing multiple sources.",
    verbose=True,
)

## Define your Tasks

In [16]:
# Define parallel research tasks
task1 = Task(
    description="Summarize a major trend in AI safety and model alignment.",
    expected_output="Concise summary of trend #1",
    agent=researcher_1,
)

task2 = Task(
    description="Summarize a key innovation in multimodal or frontier AI systems.",
    expected_output="Concise summary of trend #2",
    agent=researcher_2,
)

task3 = Task(
    description="Summarize a current topic in AI policy, regulation, or social impact.",
    expected_output="Concise summary of trend #3",
    agent=researcher_3,
)

# Aggregation task
aggregation_task = Task(
    description="Combine the three AI trend summaries into a cohesive single report.",
    expected_output="A synthesized report capturing all three trends.",
    agent=aggregator,
)

## Create Crew

In [17]:
crew = Crew(
    agents=[researcher_1, researcher_2, researcher_3, aggregator],
    tasks=[task1, task2, task3, aggregation_task],
    process=Process.sequential,
    verbose=True,
)

result = crew.kickoff()
print(result)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 5eb66231-dee3-411a-9bce-e98698969e32                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: 208eaaf8-42de-4bf8-9e5c-d233e54f6de0
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: 208eaaf8-42de-4bf8-9e5c-d233e54f6de0
       Status: Executing Task...
    └── 🤖 Agent: LLM Researcher A
            Status: In Progress

# Agent: LLM Researcher A
## Task: Summarize a major trend in AI safety and model alignment.


# Agent: LLM Researcher A
## Final Answer: 
One of the major trends in AI safety and model alignment is the shift towards "Value Alignment," which emphasizes the necessity of aligning AI systems with human values and societal norms. Researchers are increasingly focusing on ensuring that AI models do not only perform their intended tasks efficiently but also do so in a way that is consistent with ethical standards and human preferences. This entails developing methodologies and frameworks for identifying, encoding, and implementing human values within AI systems. Techniques such as Reinforcement Learning from Human Feedback (RLHF) are being utilized to better align the outcomes of AI behavior with user expectations and moral considerations. Enhanced transparency, explainability, and stakeholder engagement are also pivotal components of this trend, aiming to build trust and mitigate risks assoc

🚀 Crew: crew
└── 📋 Task: 208eaaf8-42de-4bf8-9e5c-d233e54f6de0
       Status: Executing Task...
    └── 🤖 Agent: LLM Researcher A
            Status: ✅ Completed

...

🚀 Crew: crew
├── 📋 Task: 208eaaf8-42de-4bf8-9e5c-d233e54f6de0
│      Assigned to: LLM Researcher A
│      Status: ✅ Completed
│   └── 🤖 Agent: LLM Researcher A
│           Status: ✅ Completed
├── 📋 Task: acc4d159-e585-4096-afa6-7ca86c04c608
│      Assigned to: LLM Researcher B
│      Status: ✅ Completed
│   └── 🤖 Agent: LLM Researcher B
│           Status: ✅ Completed
├── 📋 Task: f0eb346d-55d5-4373-98c9-48a2cd2ca159
│      Assigned to: LLM Researcher C
│      Status: ✅ Completed
│   └── 🤖 Agent: LLM Researcher C
│           Status: ✅ Completed
└── 📋 Task: 75b8e62b-0fc5-4d1b-837f-61f74141749e
       Status: Executing Task...
    └── 🤖 Agent: Aggregator
            Status: ✅ Completed

🚀 Crew: crew
├── 📋 Task: 208eaaf8-42de-4bf8-9e5c-d233e54f6de0
│      Assigned to: LLM Researcher A
│      Status: ✅ Completed
│   └── 🤖 Agent: LLM Researcher A
│           Status: ✅ Completed
├── 📋 Task: acc4d159-e585-4096-afa6-7ca86c04c608
│      Assigned to: LLM Researcher B
│      Status: ✅ Completed
│   └── 🤖 Agent: LLM Researcher B
│           Status: ✅ Completed
├── 📋 Task: f0eb346d-55d5-4373-98c9-48a2cd2ca159
│      Assigned to: LLM Researcher C
│      Status: ✅ Completed
│   └── 🤖 Agent: LLM Researcher C
│           Status: ✅ Completed
└── 📋 Task: 75b8e62b-0fc5-4d1b-837f-61f74141749e
       Assigned to: Aggregator
       Status: ✅ Completed
    └── 🤖 Agent: Aggregator
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 75b8e62b-0fc5-4d1b-837f-61f74141749e                                                                     │
│  Agent: Aggregator                                                                                              │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 5eb66231-dee3-411a-9bce-e98698969e32                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

The growing trend in AI safety and model alignment is known as "Value Alignment." This trend emphasizes the critical need to align AI systems with human values and societal norms. As AI technologies advance, researchers are increasingly focused on the dual objective of ensuring that AI models not only perform tasks efficiently but also adhere to ethical standards and reflect human preferences. 

Central to this trend is the development of comprehensive methodologies and frameworks designed for the identification, encoding, and implementation of human values within AI systems. Techniques like Reinforcement Learning from Human Feedback (RLHF) are being harnessed to better align AI behavioral outcomes with user expectations and moral considerations. This method allows for a feedback loop where human judgments guide the AI, enhancing the model's alignment with societal values.

Moreover, aspects like enhanced transparency, explainability, and stakeholder engagement play vital roles in this

### Check your Phoenix project to view the traces and spans from your runs.